In [1]:
from pyALT import *
import pyALT.io as io

In [2]:
G = io.load('meso_cortex.pk')

In [3]:
P = Paths(G.nodes())

In [4]:
len(P)

0

In [5]:
L = ALT(G)
thresholds = {'VISp':4,'AUDp':1,'GU':2.5,'SSs':4,'SSp-ul':2.5,'SSp-tr':2.5,
              'SSp-m':4,'SSp-bfd':4,'SSp-ll':1.5,'SSp-n':0.98,'MOB':0.46}
srcs = ['SSp-n','AUDp','VISp','GU','SSs','SSp-ul','SSp-tr','SSp-m','SSp-bfd','SSp-ll','MOB']
for src in srcs:
    L.add_source(src)
L.set_theta('MOB',0.46)

In [6]:
L.run()

In [7]:
L.form_paths()

In [12]:
L.core(90)

[('CLA', 580),
 ('PTLp', 443),
 ('AUDv', 339),
 ('AIp', 255),
 ('MOs', 203),
 ('AOB', 161),
 ('ACAd', 126),
 ('VISl', 101),
 ('ECT', 79),
 ('AId', 66),
 ('VISal', 58)]

In [19]:
L.P.P

{35: ['SSp-n', 'SSp-m', 'SSp-ul', 'SSp-ll', 'SSp-tr'],
 190: ['GU', 'SSp-m', 'SSp-ul', 'SSp-ll'],
 215: ['SSs', 'SSp-n'],
 243: ['SSs', 'SSp-ul'],
 247: ['SSs', 'SSp-bfd', 'SSp-tr'],
 250: ['SSs', 'SSp-m'],
 262: ['SSp-ul', 'SSp-m', 'SSp-n'],
 287: ['SSp-ul', 'SSs', 'GU'],
 288: ['SSp-ul', 'SSs', 'SSp-bfd'],
 340: ['SSp-tr', 'AUDp'],
 414: ['SSp-m', 'SSp-n', 'SSs', 'SSp-bfd'],
 415: ['SSp-m', 'SSs', 'SSp-bfd'],
 444: ['SSp-bfd', 'SSs', 'SSp-n'],
 474: ['SSp-bfd', 'SSp-tr'],
 477: ['SSp-bfd', 'SSp-ul', 'SSp-m'],
 515: ['SSp-ll', 'AUDp']}

In [12]:
L.P.bestNode()

TypeError: 'NoneType' object is not iterable

In [15]:
len(L)

0

In [16]:
L.run()

AttributeError: 'ALT' object has no attribute 'run'